# Import Useful Libraries

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from matplotlib import pyplot as plt
plt.rcParams["figure.facecolor"] = "white"

In [2]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher, get_close_matches
from Levenshtein import distance as levenshtein_distance
from Levenshtein import jaro as jaro_distance
from Levenshtein import jaro_winkler as jaro_winkler_distance
from fastDamerauLevenshtein import damerauLevenshtein
from fastDamerauLevenshtein import damerauLevenshtein
from stringmatch.model import *

# Load Data

In [3]:
df = pd.read_excel('../resources/financial_accounts_column_matching.xlsx')
df_ground_truth = pd.read_csv('../resources/columns_matched_reviewed_description.csv')

In [4]:
df['Non-GPO'] = df['Non-GPO'].str.split('.', 2, expand=True)[2]
df['Non-GPO'][0] = 'id'
df['Non-GPO'][1] = 'index'
df['Non-GPO'][187] = 'processed_year'
df['Non-GPO'][188] = 'processed_month'
df['Non-GPO'][189] = 'processed_day'

/var/folders/nc/6fy7qrnj4t5gn35504q8q16m0000gq/T/ipykernel_79543/2347349279.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['Non-GPO'] = df['Non-GPO'].str.split('.', 2, expand=True)[2]


In [5]:
df

,Non-GPO,GPO
0,id,id
1,index,index
2,capemployed,financials.accounts.val.date_of_accounts
3,companyclass,financials.accounts.val.currency
4,consolidatedaccts,financials.accounts.val.currency_multiplier
...,...,...
185,trnvrestworld,NaN
186,discimpaircharges,NaN
187,processed_year,NaN
188,processed_month,NaN


In [6]:
df_ground_truth = df_ground_truth.loc[(df_ground_truth['GPO'] != '?') & (df_ground_truth['GPO'] != '__event_date')]

In [7]:
df_ground_truth

,Non-GPO,GPO
15,dl12.companyname,commercial_name
16,dl12.datedissvd_dd,identification.dissolution_date
17,dl12.datedissvd_mm,identification.dissolution_date
18,dl12.datedissvd_yyyy,identification.dissolution_date
19,dl12.dateincorp_dd,identification.incorporation_date
...,...,...
803,dlb9.size_compgroupname,commercial_delphi_history.sectors.size_comp_gr...
804,dlb9.size_compsector,commercial_delphi_history.sectors.size_comp_gr...
805,dlb9.size_compsectornameend,commercial_delphi_history.sectors.size_comp_gr...
806,dlb9.size_compsectornamestart,commercial_delphi_history.sectors.size_comp_gr...


In [8]:
df_ground_truth.GPO

15                                       commercial_name
16                       identification.dissolution_date
17                       identification.dissolution_date
18                       identification.dissolution_date
19                     identification.incorporation_date
                             ...                        
803    commercial_delphi_history.sectors.size_comp_gr...
804    commercial_delphi_history.sectors.size_comp_gr...
805    commercial_delphi_history.sectors.size_comp_gr...
806    commercial_delphi_history.sectors.size_comp_gr...
808                 notice_of_correction.num_corrections
Name: GPO, Length: 587, dtype: object

# Predicting with difflib

In [9]:
matches = []

for i, x in enumerate(df_ground_truth['Non-GPO'].dropna()):
    match_list = get_close_matches(x, df_ground_truth['GPO'], cutoff=0.6)
    if match_list:
        matched = match_list[0]
    else:
        matched = ''
    matches.append((x, matched))
    
print('Done!')

Done!


In [17]:
# matches

In [11]:
df_matches = pd.DataFrame(matches, columns=['Non-GPO', 'Estimate'])
df_matches['similarity_score'] = df_matches.apply(lambda x: SequenceMatcher(None, x['Non-GPO'], x['Estimate']).ratio(), axis=1)

In [12]:
df_matches

,Non-GPO,Estimate,similarity_score
0,dl12.companyname,,0.0
1,dl12.datedissvd_dd,,0.0
2,dl12.datedissvd_mm,,0.0
3,dl12.datedissvd_yyyy,,0.0
4,dl12.dateincorp_dd,,0.0
...,...,...,...
582,dlb9.size_compgroupname,,0.0
583,dlb9.size_compsector,,0.0
584,dlb9.size_compsectornameend,,0.0
585,dlb9.size_compsectornamestart,,0.0


In [14]:
# df_athena_non_gpo.sort_values(by='similarity_score', ascending=False).to_csv('best_matches.csv', index=False)

In [15]:
df_matches['Target'] = df_ground_truth['GPO'].values

In [16]:
df_matches

,Non-GPO,Estimate,similarity_score,Target
0,dl12.companyname,,0.0,commercial_name
1,dl12.datedissvd_dd,,0.0,identification.dissolution_date
2,dl12.datedissvd_mm,,0.0,identification.dissolution_date
3,dl12.datedissvd_yyyy,,0.0,identification.dissolution_date
4,dl12.dateincorp_dd,,0.0,identification.incorporation_date
...,...,...,...,...
582,dlb9.size_compgroupname,,0.0,commercial_delphi_history.sectors.size_comp_gr...
583,dlb9.size_compsector,,0.0,commercial_delphi_history.sectors.size_comp_gr...
584,dlb9.size_compsectornameend,,0.0,commercial_delphi_history.sectors.size_comp_gr...
585,dlb9.size_compsectornamestart,,0.0,commercial_delphi_history.sectors.size_comp_gr...


In [17]:
print('Number of unique values in Estimate:', len(set(df_matches['Estimate'])))
print('Number of unique values in Target:', len(set(df_matches['Target'])))

Number of unique values in Estimate: 46
Number of unique values in Target: 551


In [18]:
accuracy_difflib = (df_matches['Target'] == df_matches['Estimate']).sum() / df_matches['Target'].shape[0]
print('Accuracy with difflib: {:.1%}'.format(accuracy_difflib))

Accuracy with difflib: 7.3%


## Using Levenshtein Distance

In [16]:
weights_score = cross_val_weights(df_ground_truth['Non-GPO'].dropna(),\
                                  df_ground_truth['GPO'],\
                                  df_ground_truth['GPO'],\
                                  algorithm='levenshtein', \
                                  max_weights=(10, 10, 10),\
                                  verbose=True)


In [26]:
df_weights_score = pd.DataFrame(weights_score, columns=['weights', 'score']).sort_values(by='score', ascending=False)

In [27]:
df_weights_score

,weights,score
89,"(1, 9, 10)",0.587734
78,"(1, 8, 9)",0.584327
79,"(1, 8, 10)",0.584327
88,"(1, 9, 9)",0.580920
99,"(1, 10, 10)",0.579216
...,...,...
603,"(7, 1, 4)",0.001704
703,"(8, 1, 4)",0.001704
904,"(10, 1, 5)",0.001704
903,"(10, 1, 4)",0.001704


In [28]:
df_weights_score.to_csv('../resources/weights/df_weights_score_lev.csv', index=False)

# Using Damerau-Levenshtein Distance

In [ ]:
weights_score = cross_val_weights(df_ground_truth['Non-GPO'].dropna(),\
                                  df_ground_truth['GPO'],\
                                  df_ground_truth['GPO'],\
                                  algorithm='damerau-levenshtein', \
                                  max_weights=(10, 10, 10, 10),\
                                  verbose=True)

In [ ]:
df_weights_score = pd.DataFrame(weights_score, columns=['weights', 'score']).sort_values(by='score', ascending=False)

In [ ]:
df_weights_score

In [ ]:
df_weights_score.to_csv('../resources/weights/df_weights_score_dam_lev.csv', index=False)

# Using Jaro Distance

In [34]:
weights_score = cross_val_weights(df_ground_truth['Non-GPO'].dropna(),\
                                  df_ground_truth['GPO'],\
                                  df_ground_truth['GPO'],\
                                  algorithm='jaro', \
                                  max_weights=0.0,\
                                  verbose=True)

9.9%


In [36]:
df_weights_score = pd.DataFrame([weights_score], columns=['score']).sort_values(by='score', ascending=False)

In [37]:
df_weights_score

,score
0,0.098807


In [38]:
df_weights_score.to_csv('../resources/weights/df_weights_score_jaro.csv', index=False)

# Using Jaro-Winkler Distance

In [10]:
weights_score = cross_val_weights(df_ground_truth['Non-GPO'].dropna(),\
                                  df_ground_truth['GPO'],\
                                  df_ground_truth['GPO'],\
                                  algorithm='jaro-winkler', \
                                  max_weights=np.linspace(0, 1, 100),\
                                  verbose=True)

0.0: 9.9%
0.010101010101010102: 9.9%
0.020202020202020204: 9.9%
0.030303030303030304: 9.9%
0.04040404040404041: 9.9%
0.05050505050505051: 9.9%
0.06060606060606061: 9.9%
0.07070707070707072: 9.9%
0.08080808080808081: 9.9%
0.09090909090909091: 9.9%
0.10101010101010102: 9.9%
0.11111111111111112: 9.9%
0.12121212121212122: 9.9%
0.13131313131313133: 9.9%
0.14141414141414144: 9.9%
0.15151515151515152: 9.9%
0.16161616161616163: 9.9%
0.17171717171717174: 9.9%
0.18181818181818182: 9.9%
0.19191919191919193: 9.9%
0.20202020202020204: 9.9%
0.21212121212121213: 9.9%
0.22222222222222224: 9.9%
0.23232323232323235: 9.9%
0.24242424242424243: 9.9%
0.25252525252525254: 9.9%
0.26262626262626265: 9.9%
0.27272727272727276: 9.9%
0.2828282828282829: 9.9%
0.29292929292929293: 9.9%
0.30303030303030304: 9.9%
0.31313131313131315: 9.9%
0.32323232323232326: 9.9%
0.33333333333333337: 9.9%
0.3434343434343435: 9.9%
0.3535353535353536: 9.9%
0.36363636363636365: 9.9%
0.37373737373737376: 9.9%
0.38383838383838387: 9.9%
0.

In [13]:
df_weights_score = pd.DataFrame(weights_score, columns=['weights', 'score']).sort_values(by='score', ascending=False)

In [14]:
df_weights_score

,weights,score
0,0.000000,0.098807
63,0.636364,0.098807
73,0.737374,0.098807
72,0.727273,0.098807
71,0.717172,0.098807
...,...,...
30,0.303030,0.098807
29,0.292929,0.098807
28,0.282828,0.098807
27,0.272727,0.098807


In [15]:
df_weights_score.to_csv('../resources/weights/df_weights_score_jaro_winkler.csv', index=False)

# Using Jaccard Simlarity
### DO NOT USE

In [ ]:
# # def jaccard_similarity(list1, list2):
# #     intersection = len(list(set(list1).intersection(list2)))
# #     union = (len(set(list1)) + len(set(list2))) - intersection
# #     return float(intersection) / union

# def calc_jaccard_similarity(string1, string2):
#     '''This function calculates the Jaccard similarity between two strings'''
#     # Jaccard similarity = A intersect B / A Union B
#     #                    = A intersect B / (A + B) - A intersect B
#     s1 = string1#.split()
#     s2 = string2#.split()
#     intersection = len(set(s1).intersection(set(s2)))
#     print('intersection:', intersection)
#     union = len(set(s1)) + len(set(s2)) - intersection
#     print('union:', union)
#     return float(intersection) / union

In [ ]:
# calc_jaccard_similarity('dl13.issuedcapcurr', 'dl13.identification.issued_capital')

In [ ]:
# # s1 = set('dl13.issuedcapcurr')
# s2 = set('dl13.identification.issued_capital')

In [23]:
type(np.linspace(0, 1, 1000)) == np.array

False